In [10]:
from neo4j import GraphDatabase

# Connect to Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "h4ck3r20o8"))

In [11]:
import neo4j
from neo4j_graphrag.indexes import create_fulltext_index

FULLTEXT_INDEX_NAME = "fulltext_index"

create_fulltext_index(
    driver, FULLTEXT_INDEX_NAME, label="Pokemon", node_properties=["description"]
)


In [ ]:
import neo4j
from neo4j_graphrag.indexes import create_vector_index

INDEX_NAME = "vector_index"
DIMENSION = 1536

driver = neo4j.GraphDatabase.driver(NEO4J_URL, auth=NEO4J_AUTH)

create_vector_index(
    driver,
    INDEX_NAME,
    label="Pokemon",
    embedding_property="vectorProperty",
    dimensions=DIMENSION,
    similarity_fn="euclidean",
)

In [ ]:
import random
from typing import Any

from neo4j_graphrag.embeddings import Embedder


class CustomEmbeddings(Embedder):
    def __init__(self, dimension: int = 10, **kwargs: Any):
        self.dimension = dimension

    def embed_query(self, input: str) -> list[float]:
        return [random.random() for _ in range(self.dimension)]


llm = CustomEmbeddings(dimensions=1024)
res = llm.embed_query("text")
print(res[:10])

In [7]:
from neo4j_graphrag.llm import LLMResponse, OpenAILLM

# not used but needs to be provided
api_key = "ollama"

llm = OpenAILLM(
    base_url="http://localhost:11434/v1",
    model_name="llama3.2:latest",
    api_key=api_key,
)

text = """
create a description about
"""

res: LLMResponse = llm.invoke(text)
print(res.content)

I'm happy to help! However, it seems like I wasn't provided with anything to create a description about. Could you please provide me with the topic or subject that you would like me to create a description for? It could be a product, service, person, place, event, or anything else. Just let me know, and I'll do my best to craft a compelling and informative description!


In [3]:
print(res.content[10:])

content=''


In [6]:
"""This example demonstrate how to embed a text into a vector
using OpenAI models and API.
"""

from neo4j_graphrag.embeddings import OpenAIEmbeddings

api_key = "ollama"

embeder = OpenAIEmbeddings(
    base_url="http://localhost:11434/v1",
    api_key=api_key,
    model="snowflake-arctic-embed:latest",
)
res = embeder.embed_query("my question")
print(len(res))

1024


In [ ]:
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.retrievers import VectorCypherRetriever

embedder = OpenAIEmbeddings(model="text-embedding-ada-002")
retrieval_query = """
    MATCH
    (actor:Actor)-[:ACTED_IN]->(node)
    RETURN
    node.title AS movie_title,
    node.plot AS movie_plot, 
    collect(actor.name) AS actors;
"""

retriever = VectorCypherRetriever(
    driver,
    index_name="moviePlotsEmbedding",
    embedder=embedder,
    retrieval_query=retrieval_query,
)
query_text = "What are the names of the actors in the movie set in 1375 in Imperial China?"
retriever_result = retriever.search(query_text=query_text, top_k=3)
print(retriever_result)